In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, to_date, count
from cassandra.cluster import Cluster

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("NewsAggregation") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000")\
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

25/01/04 00:02:49 WARN Utils: Your hostname, hadoop resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/01/04 00:02:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/04 00:03:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
yfinance_silver_path = "/user/adam_majczyk2001/nifi/silver_parquet/yfinance/"
news_silver_path = "/user/adam_majczyk2001/nifi/silver_parquet/news/"

yfinance_df = spark.read.parquet(yfinance_silver_path)
news_df = spark.read.parquet(news_silver_path)

In [3]:
# Convert `datetime` to `DATE`
news_df = news_df.withColumn("aggregation_date", to_date(col("datetime"), "yyyy-MM-dd"))

# Aggregate news data for `aggregated_news`
aggregated_news_df = news_df.groupBy(
    col("aggregation_date"), col("source_site").alias("symbol")
).agg(
    count("title").alias("total_articles")
)

# Extract keywords and aggregate for `aggregated_keywords`
keywords_df = news_df.select(
    col("aggregation_date"),
    col("source_site").alias("symbol"),
    explode(col("keywords")).alias("keyword")
).groupBy("symbol", "aggregation_date", "keyword").count()

In [5]:
aggregated_news_df.show()

+----------------+------------------+--------------+
|aggregation_date|            symbol|total_articles|
+----------------+------------------+--------------+
|      2024-12-30|            wnp.pl|            17|
|      2024-12-18|wysokienapiecie.pl|             2|
|      2024-12-15|wysokienapiecie.pl|             2|
|      2024-11-29|wysokienapiecie.pl|             2|
|      2024-11-26|            wnp.pl|            31|
|      2024-12-30|          beurs.nl|            22|
|      2024-11-30|            wnp.pl|             7|
|      2024-11-27|            wnp.pl|            33|
|      2024-12-17|            wnp.pl|             1|
|      2024-12-24|            wnp.pl|            13|
|      2024-11-23|            wnp.pl|             6|
|      2024-11-28|          beurs.nl|            28|
|      2024-10-23|wysokienapiecie.pl|             1|
|      2024-12-31|wysokienapiecie.pl|             1|
|      2024-10-15|wysokienapiecie.pl|             1|
|      2024-09-26|wysokienapiecie.pl|         

In [6]:
keywords_df.show()

+------------------+----------------+--------------------+-----+
|            symbol|aggregation_date|             keyword|count|
+------------------+----------------+--------------------+-----+
|            wnp.pl|      2024-12-30|             UKRAINA|    3|
|            wnp.pl|      2024-11-29|               UMOWY|    1|
|            wnp.pl|      2024-11-28|POLSKA GRUPA GÓRN...|    2|
|            wnp.pl|      2024-11-19|      MUSIAŁEK PAWEŁ|    1|
|            wnp.pl|      2024-11-27|     PRODUKCJA STALI|    1|
|            wnp.pl|      2024-11-23|      FARMY WIATROWE|    1|
|            wnp.pl|      2024-11-25|            GUZ ADAM|    1|
|            wnp.pl|      2024-11-26| ELEKTROWNIA ŁAZISKA|    1|
|wysokienapiecie.pl|      2024-12-21|TESTY SAMOCHODÓW ...|    1|
|            wnp.pl|      2024-11-28|            WÄRTSILÄ|    1|
|            wnp.pl|      2024-11-26|     PGE DYSTRYBUCJA|    1|
|            wnp.pl|      2025-01-02|          PREZES URE|    1|
|            wnp.pl|     

In [ ]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('gold_layer')

In [8]:
# Insert data into `aggregated_news`
for row in aggregated_news_df.collect():
    session.execute("""
        INSERT INTO aggregated_news (symbol, aggregation_date, total_articles)
        VALUES (%s, %s, %s)
    """, (row['symbol'], row['aggregation_date'], row['total_articles']))

# Insert data into `aggregated_keywords`
for row in keywords_df.collect():
    session.execute("""
        INSERT INTO aggregated_keywords (symbol, aggregation_date, keyword, count)
        VALUES (%s, %s, %s, %s)
    """, (row['symbol'], row['aggregation_date'], row['keyword'], row['count']))

print("News data aggregated and stored.")

News data aggregated and stored.


In [13]:
from pyspark.sql.functions import col, to_date, avg, max, min

# Convert `record_timestamp` to `DATE`
yfinance_df = yfinance_df.withColumn("aggregation_date", to_date(col("record_timestamp"), "yyyy-MM-dd"))

# Aggregate financial data
aggregated_yfinance_df = yfinance_df.groupBy(
    col("company").alias("symbol"), col("aggregation_date")
).agg(
    avg("price").alias("avg_price"),
    max("price").alias("max_price"),
    min("price").alias("min_price"),
    avg("volume").alias("avg_volume"),
    avg("volatility").alias("avg_volatility"),
    avg("market_sentiment").alias("avg_sentiment")
)

In [14]:
aggregated_yfinance_df.show()

+------+----------------+------------------+---------+---------+--------------------+------------------+--------------------+
|symbol|aggregation_date|         avg_price|max_price|min_price|          avg_volume|    avg_volatility|       avg_sentiment|
+------+----------------+------------------+---------+---------+--------------------+------------------+--------------------+
|   COP|      2025-01-01| 99.21789594338686|   110.58|    90.43|  3680355.0649855477|0.7912754908800959|-0.01815658327519184|
|   XOM|      2024-11-30|117.80880518059092|   127.83|   107.42|   5214347.396023349| 0.939963516964611|-0.08147537395111269|
|   COP|      2024-11-30|108.46727927595126|   116.13|   100.69|  3893415.0428518658| 0.786200960472848|-0.15081473956409325|
|   COP|      2024-12-30| 97.15079150768037|    106.9|    86.22|   3153567.389883573| 0.841118188044223| 0.00872605420213288|
|    BP|      2024-12-31| 29.28320153815817|    32.15|    26.95|   5353842.821928614|0.7821794517846573|-0.01808104910

In [16]:
# Insert data into `aggregated_yfinance`
for row in aggregated_yfinance_df.collect():
    session.execute("""
        INSERT INTO aggregated_yfinance (symbol, aggregation_date, avg_stock_price, max_stock_price, min_stock_price, volume_traded, avg_volatility, avg_sentiment)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (row['symbol'], row['aggregation_date'], row['avg_price'], row['max_price'], row['min_price'], row['avg_volume'], row['avg_volatility'], row['avg_sentiment']))

print("YFinance data aggregated and stored.")

YFinance data aggregated and stored.


In [ ]:
# Stop Spark Session
spark.stop()